In [12]:
import osmnx as ox, geopandas as gpd
from shapely.geometry import Point, MultiLineString

import os
BASE_DIR = os.path.dirname(os.getcwd())
os.chdir(BASE_DIR + '/src/data/')
import util
os.chdir(BASE_DIR + '/notebooks/')

In [2]:
gdf_place = ox.gdf_from_place('Cambridge, Massachusetts, USA')
polygon = gdf_place['geometry'].unary_union
cambridge = ox.core.osm_net_download(polygon)

In [18]:
elements = cambridge[0]['elements']

# Grab all the ways in a list
ways = [x for x in elements if x['type'] == 'way']

# Turn the nodes into key value pairs where key is the node id
nodes = {x['id']: x for x in elements if x['type'] == 'node'}

i = 0
# Create multiline strings for all ways
for way in ways:
    if i < 10:
        print way
        print nodes[way['nodes'][0]]
    i += 1

{u'nodes': [61171565, 61171566, 61171503, 61171502, 61171508, 61171505, 61172198, 61171615], u'type': u'way', u'id': 8604746, u'tags': {u'lanes': u'2', u'name': u'Columbia Street', u'width': u'12.2', u'created_by': u'JOSM', u'source': u'massgis_import_v0.1_20071008141127', u'highway': u'residential', u'attribution': u'Office of Geographic and Environmental Information (MassGIS)', u'massgis:way_id': u'193222', u'condition': u'deficient'}}
{u'lat': 42.374253, u'lon': -71.093746, u'type': u'node', u'id': 61171565}
{u'nodes': [61324188, 61322026, 61322009, 267656464, 1530477573, 1530477577, 61322007, 267656466, 278949282, 61322006, 61322003], u'type': u'way', u'id': 8604813, u'tags': {u'lanes': u'3', u'name': u"Monsignor O'Brien Highway", u'width': u'30.5', u'source': u'massgis_import_v0.1_20071008165629', u'condition': u'fair', u'oneway': u'yes', u'attribution': u'Office of Geographic and Environmental Information (MassGIS)', u'massgis:way_id': u'159753', u'ref': u'MA 28', u'highway': u't

In [20]:



node_counts = {}
dup_nodes = {}
# Any node that's shared by more than one way is an intersection

j = 0
way_lines = []
for way in ways:
    coords = []
    way_nodes = way['nodes']
    #for n in way['nodes']:
    prev = None
    for i in range(len(way_nodes)):
        n = way_nodes[i]
        if prev:
            coords.append((
            (prev['lon'], prev['lat']),
            (nodes[n]['lon'], nodes[n]['lat']),
        ))
        prev = nodes[n]

        if n in node_counts.keys():
            dup_nodes[n] = 1

        else:
            node_counts[n] = 0
        node_counts[n] += 1
    # Make the multiline string for this way
    if j < 5:
        print coords
    j += 1
    way_lines.append(MultiLineString(coords))

print way_lines[0]
print "Found " + str(len(dup_nodes.keys())) + " intersections"
print nodes.values()[0]

[((-71.093746, 42.374253), (-71.09386, 42.374701)), ((-71.09386, 42.374701), (-71.094018, 42.375226)), ((-71.094018, 42.375226), (-71.094088, 42.375508)), ((-71.094088, 42.375508), (-71.094108, 42.375592)), ((-71.094108, 42.375592), (-71.094234, 42.375782)), ((-71.094234, 42.375782), (-71.094708, 42.375808)), ((-71.094708, 42.375808), (-71.095239, 42.375868))]
[((-71.0755329, 42.3705222), (-71.0759287, 42.3708229)), ((-71.0759287, 42.3708229), (-71.0761423, 42.3709661)), ((-71.0761423, 42.3709661), (-71.0763655, 42.3711056)), ((-71.0763655, 42.3711056), (-71.0764908, 42.3711798)), ((-71.0764908, 42.3711798), (-71.0766269, 42.3712544)), ((-71.0766269, 42.3712544), (-71.076771, 42.371325)), ((-71.076771, 42.371325), (-71.076939, 42.3713971)), ((-71.076939, 42.3713971), (-71.0771155, 42.3714679)), ((-71.0771155, 42.3714679), (-71.0775623, 42.3716339)), ((-71.0775623, 42.3716339), (-71.0777611, 42.3717095))]
[((-71.087917, 42.373499), (-71.087911, 42.373517)), ((-71.087911, 42.373517), (-7

In [23]:
# Write all the ways out
schema = {'geometry': 'MultiLineString', 'properties': {}}
util.write_shp(schema, BASE_DIR + '/osm-data/processed/maps/ways.shp', way_lines, 0, 1)
print BASE_DIR

points = []

for node in dup_nodes.keys():
    points.append((
        Point(nodes[node]['lon'], nodes[node]['lat']),
        {'node_id': node}
    ))
schema = {'geometry': 'Point', 'properties': {'node_id': 'int'}}
util.write_points(points, schema, BASE_DIR + '/osm-data/processed/maps/osm_inter.shp')

/home/jenny/boston-crash-modeling
finished 0 of 4535
finished 500 of 4535
finished 1000 of 4535
finished 1500 of 4535
finished 2000 of 4535
finished 2500 of 4535
finished 3000 of 4535
finished 3500 of 4535
finished 4000 of 4535
finished 4500 of 4535
